## To Do List
#### 1. For Data
* Check if the tokenizer does enough cleaning or not
* Remove contractions
* Use paraphrasing model to generate an equally sized dataset and encode it and save its representations
* Use Huawei Augmenter on TinyBERT to generate an equally sized data and save its representations

#### 3. For Training
* Train 50 epochs using Linear Probing
* Generate Multi-nomial predictions
* Compare argmax vs decision Tree

* Train 50 epochs using the whole model
* Generate Multi-nomial predictions
* Compare argmax vs decision Tree

* Train 2 Epochs on the whole dataset
* Generate Multi-nomial predictions
* Compare argmax vs decision Tree

#### 4. For Inference
* Pick-up the best model, the best cleaning technique, and the best Inference technique to do the inference

# 1. Expriment Design

### 1. Model Selection
##### 1.1. What are the questions to ask about the model selection:
1. Which model to be used?
According to papers with code the SOTA on Yelp dataset is a variation of RoBERTa. Therefore RoBERTa is a decent architecture to start with
2. fined-tuned of RoBERTa or Agnostic?
Since Yelp fine-tuned RoBERTa is not allowed, the two options available to test are Amazon-Polarity fine-tuned RoBERTa and agnostic RoBERTa
3. How to select between these two models with no expensive training?
Using both Models to generate representations, and Run a decision tree on the top of them as a way to compare the quality of representations
4. Why decision Trees?
After running a punch of trials, the confusion matrix reflected that the main drop in accuracy takes place due to the confusion between (2, 3) and (1, 2). Therefore, I think that argmax is not the most efficient way to pick up the threshold of classification, and picking up thresholds that reduce entropy is the speciality of decision trees

##### 1.2. Results: 
To be presented

### 2. Data Nature
##### 2.1. What are the questions to ask about the data:
**Data Quality:**
1. How to know if the data needs preprocessing?
* Tokenize it then un-tokenize it and compare the differences and the quality of the tokenizer
* Compare the maximum length of tokens to 512 (to check if it will fit the model)
* remove the contractions as additional data samples
* clean the text manually then run decision tree again to check if the cleaning improved the representation

**Data Quantity:**
1. How to do data Augmentation?
* Using paraphrasing model to generate more examples
* Using API-based LLMs to generate more samples
* Using TinyBERT technique to do data augmentation

### 3. Training Techniques
##### 3.1.  What are the questions to ask about the Training:
1. What is the training procedure?
* Linear probing for the early training phase for 50 epochs and save lowest testing loss point (Or lowest f1 score??)
* Training the whole model for another 50 epochs with the training set and and pick the lowest validation loss
* running 2 epochs on the whole data set
2. What are the hyper-parameters to tune?
* learning-rate
* no. of epochs
* Optimizer: Adam
* Batch size: 16
* 

3. How to evaluate the training?
* Confusion Matrix

### 4. Inference Technique:
##### 4.1. What are the questions to ask about the Inference:
1. How to determine the predicted label from the generated multi-nomial distribution?
* Train a decision Tree on the top of the multinomial to setup the thresholds to correct the model bias

# 0. Importing dependencies & Configurations

In [1]:
import data
import models
import training

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import random
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification

import string
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

C:\Users\Hussein\anaconda3\envs\KD_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hussein\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hussein\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hussein\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
random.seed(42)

torch.cuda.empty_cache()
gc.collect()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.autograd.set_detect_anomaly(True)
print(device)

cuda:0


# 1. Main Parameters

In [4]:
DATA_PATH = "../stat940-winter-2024-datachallenge2/"
PRETRAINED_MODELS = {
    "amazon" : "pig4431/amazonPolarity_roBERTa_5E", 
    "twitter" : "cardiffnlp/twitter-roberta-base-emotion"
}
CHECKPOINT = None
INITIALIZATION = None

# 2. Data, Dataloader & Tokenizer

### 2. Data Nature
##### 2.1. What are the questions to ask about the data:
**Data Quality:**
1. How to know if the data needs preprocessing?
* Tokenize it then un-tokenize it and compare the differences and the quality of the tokenizer
* Compare the maximum length of tokens to 512 (to check if it will fit the model)
* remove the contractions as additional data samples
* clean the text manually then run decision tree again to check if the cleaning improved the representation

**Data Quantity:**
1. How to do data Augmentation?
* Using paraphrasing model to generate more examples
* Using API-based LLMs to generate more samples
* Using TinyBERT technique to do data augmentation

## 2.1. Testing the quality of the Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODELS["amazon"])

In [6]:
dataset = pd.read_csv("../stat940-winter-2024-datachallenge2/train.csv")

In [7]:
stats = {
    "index": [],
    "length": [],
    "label": []
}
max_length = 0
index = 0
for data_point in range(0, len(dataset["text"])):
    x, y = dataset["text"][data_point], dataset["stars"][data_point]
    tokenized_x = tokenizer(x)
    z = len(tokenized_x["input_ids"])
    stats["index"].append(index)
    stats["length"].append(z)
    stats["label"].append(y.item()+1)
    index+=1
stats = pd.DataFrame(stats)

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


In [8]:
# How many long comments
print(f"The number of long sentences = ", len(stats[stats['length']>512]))
# The distribution of long comments over the labels
print("The distribution of long comments over the labels")
print(stats[stats["length"]>512].groupby("label")["index"].count())

The number of long sentences =  198
The distribution of long comments over the labels
label
2    70
3    67
4    61
Name: index, dtype: int64


## 2.2. Data Preprocessing

In [9]:
#shuffling
dataset = dataset.sample(frac=1).reset_index(drop=True)#.to_csv("../stat940-winter-2024-datachallenge2/shuffled_train.csv", index_label='ID')

In [10]:
def basic_cleaning(text):
    text = re.sub(r'[\n\t\r]', ' ', text)
    return text

In [11]:
dataset["text"] = dataset["text"].apply(basic_cleaning)

### 2.2.1 Text Cleaning

In [12]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Expand contractions
    text = contractions.fix(text)
    # Remove special characters and numbers using regex
    text = re.sub(r'[^a-zA-Z.?!\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    #stop_words = set(stopwords.words('english'))
    #tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

In [13]:
dataset["cleaned_text"] = dataset["text"].apply(clean_text)

### 2.2.1 Data Augmentation

Different Used Data Augmentation Techniques:
1. Text Summarization (Expensive but Important)

from transformers import pipeline
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

2. Synonym Augmentation (Cheap & Important)

aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.1)
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

3. Contextual Word Embeddings Augmenter (Expensive But Important)

print("Contextual Word Embeddings Augmenter")
#Insert word by contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet)
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", aug_p=0.5, aug_min=10)
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

4. Sentence Paraphrasing


#### 2.2.1.1 Text Contextual Augmentation

In [19]:
context_aug = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute", aug_min=10, aug_max=50, device="cuda")

def embeddings_augmenter(text):
    return context_aug.augment(text)


In [20]:
dataset["context_text"] = dataset["text"].apply(embeddings_augmenter)

#### 2.2.1.2 Text Synonimization

In [16]:
syn_aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.1)

def syn_augmenter(text):
    return syn_aug.augment(text)

In [17]:
dataset["syn_text"] = dataset["text"].apply(syn_augmenter)

In [21]:
dataset.to_csv("../stat940-winter-2024-datachallenge2/shuffled_train.csv", index_label='ID')

#### 2.2.1.3 Text Summarization

In [22]:
from transformers import pipeline
pipe = pipeline("summarization", model="facebook/bart-large-cnn", device=0, )

def text_summarization(text):
    return pipe(text)[0]["summary_text"]

In [23]:
dataset["summary_text"] = dataset["text"].apply(text_summarization)

Your max_length is set to 142, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 142, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
C:\Users\Hussein\anaconda3\envs\KD_torch\lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 142, but your input_length is only 118. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Your max_length 

Your max_length is set to 142, but your input_length is only 98. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 142, but your input_length is only 108. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)
Your max_length is set to 142, but your input_length is only 131. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)
Your max_length is set to 142, but your input_length is only 66. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)
Yo

Your max_length is set to 142, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)
Your max_length is set to 142, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 142, but your input_length is only 141. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)
Your max_length is set to 142, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
You

Your max_length is set to 142, but your input_length is only 112. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Your max_length is set to 142, but your input_length is only 91. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 142, but your input_length is only 134. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)
Your max_length is set to 142, but your input_length is only 118. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Y

Your max_length is set to 142, but your input_length is only 102. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
Your max_length is set to 142, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 142, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 142, but your input_length is only 114. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Yo

...

Your max_length is set to 142, but your input_length is only 132. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 142, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Your max_length is set to 142, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Your max_length is set to 142, but your input_length is only 100. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Your max_length is set to 142, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 142, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 142, but your input_length is only 129. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)
Your max_length is set to 142, but your input_length is only 132. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Yo

Your max_length is set to 142, but your input_length is only 118. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Your max_length is set to 142, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
Your max_length is set to 142, but your input_length is only 132. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 142, but your input_length is only 100. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Your max_length is set to 142, but your input_length is only 141. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)
Your max_length is set to 142, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 142, but your input_length is only 131. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)
Your max_length is set to 142, but your input_length is only 114. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Y

Your max_length is set to 142, but your input_length is only 134. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)


## 2.3. Writing Data Back 

In [24]:
dataset.to_csv("../stat940-winter-2024-datachallenge2/shuffled_train.csv", index_label='ID')

In [25]:
dataset["summary_text"][0]

"We haven't been here I a couple of years because it was terrible the last time. The chips were ok, salsa is fantastic. I ordered a margarita and it tasted good. It was so packed with ice there was nothing in the glass. I had Tacos pastor. The pork was dry and the tortillas ice cold and dried out. Husband had a quesadilla because he didn't like the menu."

#### 2.2.1.4 Deep Text Summarization

In [26]:
def deep_summarization(text):
    return pipe(text, max_length=100)[0]["summary_text"]

In [27]:
dataset["deepSummary_text"] = dataset["text"].apply(deep_summarization)

C:\Users\Hussein\anaconda3\envs\KD_torch\lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 100, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 100, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)
Your max_length is set to 100, but your input_length is only 81. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)
Your max_length is

Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 67. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)
Your max_length is set to 100, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)
Your max_length is set to 100, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your

Your max_length is set to 100, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 100, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 100, but your input_length is only 72. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your

Your max_length is set to 100, but your input_length is only 81. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)
Your max_length is set to 100, but your input_length is only 97. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)
Your max_length is set to 100, but your input_length is only 92. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your

Your max_length is set to 100, but your input_length is only 74. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)
Your max_length is set to 100, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 100, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)
Your

...

Your max_length is set to 100, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 100, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 100, but your input_length is only 77. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 100, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your

Your max_length is set to 100, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 100, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 100, but your input_length is only 60. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)
Your max_length is set to 100, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)
Your

Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 100, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 100, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 100, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)
Your

Your max_length is set to 100, but your input_length is only 77. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your max_length is set to 100, but your input_length is only 64. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your

Your max_length is set to 100, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 100, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 100, but your input_length is only 95. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 91. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your

In [28]:
dataset.to_csv("../stat940-winter-2024-datachallenge2/shuffled_train_1.csv", index_label='ID')